In [ ]:
# Question: Data Quality Automation Tools - Introduction to Great Expectations
# Description: Set up a simple Great Expectations check for missing values in a numeric column.



In [2]:
pip install great_expectations


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 34.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━

In [4]:
import os
import pandas as pd
import great_expectations as ge
from great_expectations.data_context import get_context

# Step 1: Set up directory
project_dir = "my_gx_project"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

# Step 2: Initialize Great Expectations programmatically
context = ge.get_context()

# Step 3: Create sample CSV data
df = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "value": [10, 15, None, 20]
})
csv_path = os.path.abspath("sample_data.csv")
df.to_csv(csv_path, index=False)

# Step 4: Create Expectation Suite
suite_name = "missing_values_suite"
context.create_expectation_suite(expectation_suite_name=suite_name, overwrite_existing=True)

# Step 5: Define a BatchRequest for this CSV file
batch_request = {
    "datasource_name": "my_datasource",
    "data_connector_name": "default_runtime_data_connector_name",
    "data_asset_name": "sample_data_asset",
    "runtime_parameters": {"path": csv_path},
    "batch_identifiers": {"default_identifier_name": "default_id"}
}

# Register a simple Pandas-based datasource
context.add_datasource(
    name="my_datasource",
    class_name="Datasource",
    execution_engine={"class_name": "PandasExecutionEngine"},
    data_connectors={
        "default_runtime_data_connector_name": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"]
        }
    }
)

# Step 6: Create a Validator and add expectation
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=suite_name
)

# Expect no missing values in 'value' column
validator.expect_column_values_to_not_be_null("value")

# Save expectation suite
validator.save_expectation_suite(discard_failed_expectations=False)

# Step 7: Run a validation
checkpoint_name = "missing_value_checkpoint"
checkpoint_config = {
    "name": checkpoint_name,
    "config_version": 1.0,
    "class_name": "SimpleCheckpoint",
    "validations": [{"batch_request": batch_request, "expectation_suite_name": suite_name}],
}
context.add_or_update_checkpoint(**checkpoint_config)

results = context.run_checkpoint(checkpoint_name=checkpoint_name)

# Step 8: Output validation result summary
validation_result = results["run_results"]
for run_key, result in validation_result.items():
    print("Validation Success:", result["validation_result"]["success"])
    print("Expectations Met:", result["validation_result"]["statistics"])


ModuleNotFoundError: No module named 'numpy.rec'